In [ ]:
# Install the required packages
%pip install pandas Office365-REST-Python-Client
%pip install pyyaml

In [ ]:
%pip install openpyxl

In [ ]:
# Import the required libraries
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
import io
import pandas as pd

# Import the utils library
import sys
sys.path.append("..")
from utils.filesystem import *


In [ ]:
import json

# Config file path
config_file = '../config.json'

# Load the config file
with open(config_file) as f:
    config = json.load(f)

# Load the config parameters
name_mapping = config['name_mapping']
columns_to_keep = config['columns_to_keep']
site_url = config['site_url']
username = config['username']
password = config['password']
input_file_path = config['file_path']
local_path = config['local_path']
file_name = config['file_name']
sheet_name = config['sheet_name']


In [ ]:
# Ignore UserWarning messages
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Load a specific worksheet from the Excel file
try:
    dataframe = load_excel_sheet_to_dataframe (local_path, file_name, sheet_name, 4)

    # Display the first 5 rows of the dataframe
    print(f"Dataframe has lenght {len(dataframe)}")

    # Display the first 5 rows of the dataframe
    print(dataframe.head())
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
# Rename the columns
dataframe = dataframe.rename(columns=name_mapping)

# Calculate the columns to drop as the difference between all columns and the columns to keep
columns_to_drop = dataframe.columns.difference(columns_to_keep)

# Drop the columns
dataframe = dataframe.drop(columns=columns_to_drop)

# Display the first 5 rows of the dataframe
print(dataframe.head())